<a href="https://colab.research.google.com/github/jimmytju/AHF-Rapid-Diagnosis/blob/main/Yaseen_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import google.drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Tue Nov 15 15:46:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import database

In [3]:
!unzip -q /content/drive/MyDrive/Yaseen-HS/dataset-Yaseen.zip 
!git clone https://github.com/jimmytju/AHF-Rapid-Diagnosis.git

Cloning into 'AHF-Rapid-Diagnosis'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


# Import tools

In [16]:
import os,sys
sys.path.append('/content/AHF-Rapid-Diagnosis')
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from datetime import datetime
from tqdm import tqdm
import sys
import torch.optim as optim
import argparse
import  os, glob
import  random, csv
from    PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from utils import train2
from utils import plot_history
import librosa
import pandas as pd
import math
import pywt
torch.__version__

'1.12.1+cu113'

# Dataset_Dataloader

In [17]:
def Normal(list):
    new_list=[]
    max_value = max(np.abs(list))
    for i in list:
        new_list.append(i/max_value)
    return new_list

def threshold_a(num,t,a):
    if -t<num<t:
        result=0
    elif num>=t:
        result = a/2*(math.e**((num-t)/a)-math.e**((t-num)/a))    
    else:
        result = a/2*(math.e**((num+t)/a)-math.e**((-t-num)/a)) 
    return result
def self_threshold(audio_list,t,a):
    new_list = []
    t = t*max(np.abs(audio_list))
    for audio in audio_list:
#         print(audio)
        audio = threshold_a(audio,t,a)
#         print(audio)
        new_list.append(audio)
    return np.array(new_list)

def pywavelets_denoise(signal,sr,threshold=0.2,mode='self',t=0.2,a=3):
    coeffs = pywt.wavedec(signal,'sym8',level=7) # 将信号进行小波分解
    if mode == 'self':
        for i in range(1, len(coeffs)):
            coeffs[i] = self_threshold(audio_list=coeffs[i],t=t,a=a)
    else:
        for i in range(1, len(coeffs)):
            coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]),mode=mode) 
    datarec = pywt.waverec(coeffs, 'sym8') # 将信号进行小波重构
    return datarec

def dataprocess(data,sr):
  audio_data=pywavelets_denoise(data)
  audio_data = samplerate.resample(audio_data.T, 1000 / sr, converter_type='sinc_best').T
  audio_data = audio_data/np.max(np.abs(audio_data))
  return audio_data

n_fft = 2048
win_length = None
hop_length = 512
n_mels = 256
n_mfcc = 256
def mfcc(audio, sampling_rate):
    melspec = librosa.feature.melspectrogram(
    y=audio,
    sr=sampling_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    n_mels=n_mels,
    htk=True,
    norm=None,)
    mfcc_librosa = librosa.feature.mfcc(
    S=librosa.core.spectrum.power_to_db(melspec),
    n_mfcc=n_mfcc,
    dct_type=2,
    norm="ortho",)
    mfcc_librosa = librosa.power_to_db(mfcc_librosa)
    return mfcc_librosa

In [20]:
class PCGload(Dataset):
    def __init__(self, root, resize, label):
        super(PCGload, self).__init__()

        self.root = root
        self.resize = resize
        self.name2label = label # "n":0  "abn":1
        self.images, self.labels = self.load_csv('images.csv')

    def load_csv(self, filename):

        if not os.path.exists(os.path.join(self.root, filename)):
            images = []
            for name in self.name2label.keys():
                images += glob.glob(os.path.join(self.root, name, '*.wav'))

            print(len(images), images)

            random.shuffle(images)
            with open(os.path.join(self.root, filename), mode='w', newline='') as f:
                writer = csv.writer(f)
                for img in images: # 'pokemon\\bulbasaur\\00000000.png'
                    name = img.split(os.sep)[-2]
                    label = self.name2label[name]
                    # 'pokemon\\bulbasaur\\00000000.png', 0
                    writer.writerow([img, label])
                print('writen into csv file:', filename)

        # read from csv file
        images, labels = [], []
        with open(os.path.join(self.root, filename)) as f:
            reader = csv.reader(f)
            for row in reader:
                # 'pokemon\\bulbasaur\\00000000.png', 0
                img, label = row
                label = int(label)

                images.append(img)
                labels.append(label)

        assert len(images) == len(labels)

        return images, labels



    def __len__(self):

        return len(self.images)

    def __getitem__(self, idx):

        wav, label = self.images[idx], self.labels[idx]

        audio_data,fs=librosa.load(wav)
        audio_data = pywavelets_denoise(signal=audio_data,sr = fs)
        mfcc_to_save = mfcc(audio_data, fs)

        tf = transforms.Compose([
            lambda x:Image.open(x).convert('RGB'), # string path= > image data
            transforms.Resize((int(self.resize*1.25), int(self.resize*1.25))),
            transforms.RandomRotation(15),
            transforms.CenterCrop(self.resize),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

        img = tf(mfcc_to_save)
        label = torch.tensor(label)


        return  img,label

In [44]:
audio_data,fs=librosa.load('/content/dataset-Yaseen/n-as/test/abn/New_AS_002.wav')
audio_data = pywavelets_denoise(signal=audio_data,sr = fs)
mfcc_to_save = mfcc(audio_data, fs)

mfcc_to_save = torch.from_numpy(mfcc_to_save)
tf = transforms.Compose([
    transforms.CenterCrop(60),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

img = tf(mfcc_to_save)
plt.imshow(img)
plt.imsave('./0.png',img)

ValueError: ignored

In [46]:
type(mfcc_to_save)

torch.Tensor

# Training

In [10]:
class BasicBlock(nn.Module):
    """
    对于浅层网络，如ResNet-18/34等，用基本的Block
    基础模块没有压缩,所以expansion=1
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True),
            nn.Conv2d(out_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        # 如果输入输出维度不等，则使用1x1卷积层来改变维度
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels),
            )
    def forward(self, x):
        out = self.features(x)
#         print(out.shape)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out
class Bottleneck(nn.Module):
    """
    对于深层网络，我们使用BottleNeck，论文中提出其拥有近似的计算复杂度，但能节省很多资源
    zip_channels: 压缩后的维数，最后输出的维数是 expansion * zip_channels
    针对ResNet50/101/152的网络结构,主要是因为第三层是第二层的4倍的关系所以expansion=4
    """
    expansion = 4
    
    def __init__(self, in_channels, zip_channels, stride=1):
        super(Bottleneck, self).__init__()
        out_channels = self.expansion * zip_channels
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, zip_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, zip_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        out = self.features(x)
#         print(out.shape)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out
class ResNet(nn.Module):
    """
    不同的ResNet架构都是统一的一层特征提取、四层残差，不同点在于每层残差的深度。
    对于cifar10，feature map size的变化如下：
    (32, 32, 3) -> [Conv2d] -> (32, 32, 64) -> [Res1] -> (32, 32, 64) -> [Res2] 
 -> (16, 16, 128) -> [Res3] -> (8, 8, 256) ->[Res4] -> (4, 4, 512) -> [AvgPool] 
 -> (1, 1, 512) -> [Reshape] -> (512) -> [Linear] -> (10)
    """
    def __init__(self, block, num_blocks, num_classes=2, verbose = False):
        super(ResNet, self).__init__()
        self.verbose = verbose
        self.in_channels = 64
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        #使用_make_layer函数生成上表对应的conv2_x, conv3_x, conv4_x, conv5_x的结构
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # cifar10经过上述结构后，到这里的feature map size是 4 x 4 x 512 x expansion
        # 所以这里用了 4 x 4 的平均池化
        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.classifer = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        # 第一个block要进行降采样
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            # 如果是Bottleneck Block的话需要对每层输入的维度进行压缩，压缩后再增加维数
            # 所以每层的输入维数也要跟着变
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.features(x)
        if self.verbose:
            print('block 1 output: {}'.format(out.shape))
        out = self.layer1(out)        
        if self.verbose:
            print('block 2 output: {}'.format(out.shape))
        out = self.layer2(out)
        if self.verbose:
            print('block 3 output: {}'.format(out.shape))
        out = self.layer3(out)
        if self.verbose:
            print('block 4 output: {}'.format(out.shape))
        out = self.layer4(out)
        if self.verbose:
            print('block 5 output: {}'.format(out.shape))
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.classifer(out)
        return out

In [12]:
def ResNet18(verbose=False):
    return ResNet(BasicBlock, [2,2,2,2],verbose=verbose)
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 判断是否用GPU

In [19]:
epoch = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu' # 判断是否用GPU
net = ResNet18().to(device)
if device == 'cuda':
    net = nn.DataParallel(net)
# 当计算图不会改变的时候（每次输入形状相同，模型不改变）的情况下可以提高性能，反之则降低性能
torch.backends.cudnn.benchmark = True
optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5,verbose=True,patience = 5,min_lr = 0.000001) # 动态更新学习率
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)
label = {'n': 0, 'abn': 1}
Batch_Size = 256
train_root = '/content/dataset-Yaseen/n-as/train'
test_root = '/content/dataset-Yaseen/n-as/test'
train_db = PCGload(train_root, 32, label)
test_db = PCGload(test_root, 32, label)
trainloader = DataLoader(train_db, batch_size=Batch_Size, shuffle=True,num_workers=0)
testloader = DataLoader(test_db, batch_size=Batch_Size, num_workers=0)
save_path = './model/ResNet.pth'
Acc, Loss, Lr = train2(net, trainloader, testloader, epoch, optimizer, criterion, scheduler, save_path, verbose = True)

AttributeError: ignored

# Git save

In [5]:
!pip install google-colab-shell
from google_colab_shell import getshell

getshell()
getshell(height=400)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4124 sha256=41a979c86d58e7c93a22eb3e0225c524a40d60d0e73c9a69a0fde6419176f2eb
  Stored in directory: /root/.cache/pip/wheels/ee/62/db/d033316a546102d1da60b51a0cc4ddcf0b8c3cc6044469a47b
Successfully built google-colab-shell
